In [9]:
# import streamlit as st
import qiskit
import numpy as np 
import matplotlib.pyplot as plt
import copy
import qiskit_nature
print(qiskit_nature.__version__)

0.7.1


In [10]:
from qiskit import QuantumCircuit, BasicAer
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit.utils import QuantumInstance
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SLSQP
from qiskit_nature.converters.second_quantization.qubit_op_to_qubit_op import HartreeFock
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import UCCSD

from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit_nature.transformers import FreezeCoreTransformer

ModuleNotFoundError: No module named 'qiskit_nature.converters'

In [ ]:





molecule = 'H .0 .0 -{0}; Li .0 .0 {0}'
distances = np.arange(0.5, 4.25, 0.25)
vqe_energies = []
energies = []
exact_energies = []

for i, d in enumerate(distances):
    print('step', i)
    # set up experiment
    driver = PySCFDriver(molecule.format(d/2), basis='sto3g')
    problem = ElectronicStructureProblem(driver, [FreezeCoreTransformer(remove_orbitals=[3, 2])])

    # Generate the second-quantized operators
    second_q_ops = problem.second_q_ops()

    # Hamiltonian
    main_op = second_q_ops[0]

    # Setup the mapper and qubit converter
    mapper = ParityMapper()
    converter = QubitConverter(mapper=mapper, two_qubit_reduction=True)

    # The fermionic operators are mapped to qubit operators
    num_particles = (problem.molecule_data_transformed.num_alpha,
                 problem.molecule_data_transformed.num_beta)
    num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals
    qubit_op = converter.convert(main_op, num_particles=num_particles)

    init_state = HartreeFock(num_spin_orbitals, num_particles, converter)

    # setup the classical optimizer
    optimizer = SLSQP(maxiter=1000)

    # setup the variational form for VQE
    var_form = UCCSD(converter,num_particles,num_spin_orbitals,initial_state = init_state)

    # setup and run VQE
    vqe = VQE(ansatz=var_form, optimizer=optimizer, quantum_instance=BasicAer.get_backend('statevector_simulator'))
    vqe_result = np.real(vqe.compute_minimum_eigenvalue(qubit_op).eigenvalue)
    vqe_energies.append(vqe_result)

    # exact classical result
    exact_result = NumPyMinimumEigensolver(main_op).run()
    exact_energies.append(np.real(exact_result.eigenvalue))

plt.plot(distances, exact_energies, label="Exact Energy")
plt.plot(distances, vqe_energies, label="VQE Energy")
plt.xlabel('Atomic distance (Angstrom)')
plt.ylabel('Energy')
plt.legend()
plt.show()